In [1]:
import numpy as np
import pandas as pd

### Synthesize the data

In [2]:
# Parameters for the synthetis of data
n = 50000
num_cont_features = 30
num_cat_features = 10
num_cont_relevant = 4
dy = 1 
train_size = int(.66 * n)
val_size = int(.15 * n)
test_size = n - train_size - val_size

In [3]:
m = 5
x0 = np.random.randint(m, size=(n, 1))
x1 = np.random.randint(m, size=(n, 1))
others = np.random.randint(m, size=(n, num_cat_features - 2))
all_ = np.concatenate(
    [x0, x1, others],
    axis=1
)
chooser = np.asarray(x0[:, 0] == x1[:, 0], dtype=int) # k + x0 - x1 # np.asarray(x0 == x1, dtype=int)
permuter =  np.random.permutation(np.eye(num_cat_features, dtype=int).T).T
xcat = np.array(all_ @ permuter, dtype=int)
expected_cat = sorted([np.argmax(permuter[0, :]), np.argmax(permuter[1, :])])

In [4]:
xcont = np.random.uniform(size=(n, num_cont_features))
expected0 = np.sort(np.random.choice(num_cont_features, replace=False, size=num_cont_relevant))
expected1 = np.sort(np.random.choice(num_cont_features, replace=False, size=num_cont_relevant))
exps = [expected0, expected1]

In [5]:
expected_cont = list(num_cat_features + np.sort(np.array(list(set(list(expected0) + list(expected1))))))
t0 = np.random.uniform(low=-1., high=1., size=(1, dy, num_cont_relevant))
t1 = np.random.uniform(low=-1., high=1., size=(1, dy, num_cont_relevant))
ts = [t0, t1]
y = np.zeros(shape=(n, dy))
idx0 = chooser == 0
idx1 = chooser == 1
y[idx0, :] = (t0 @ np.expand_dims(np.sin(2 * np.pi * xcont[idx0][:, expected0]), axis=2))[:, :, 0]
y[idx1, :] = (t1 @ np.expand_dims(np.cos(2 * np.pi * xcont[idx1][:, expected1]), axis=2))[:, :, 0]

In [6]:
x = np.concatenate((xcat, xcont), axis=1)

In [7]:
expected_features = expected_cat + expected_cont
feature_cols = [f'x{n}' for n in range(num_cat_features + num_cont_features)]
cat_features = feature_cols[:num_cat_features]
float_features = feature_cols[num_cat_features:]
targets = [f'y{n}' for n in range(dy)]
targets = targets
xdfcat = pd.DataFrame(
    xcat,
    columns=cat_features
)
xdfcont = pd.DataFrame(
    xcont,
    columns=float_features
)
xdf = pd.concat((xdfcat, xdfcont), axis=1)
ydf = pd.DataFrame(
    y,
    columns=targets
)
data = pd.concat((xdf, ydf), axis=1)
train_data = data.iloc[:train_size]
val_data = data.iloc[train_size: train_size + val_size]
test_data = data.iloc[train_size + val_size:]